# 軽量ネットワークとMobileNet

複雑なネットワークは、学習や高速な推論のために、GPUなどの膨大な計算資源を必要とすることがわかりました。しかし、ほとんどの場合、パラメータの数が大幅に少ないモデルでも、それなりの性能を発揮できるように学習できることがわかりました。言い換えれば、モデルの複雑さが増しても、モデルの性能は比例しないで小さくなるのが一般的です。

このことは、モジュールの初期にMNISTの数字分類を学習したときに観察されました。単純な密なモデルの精度は、強力なCNNの精度よりも大きくは悪くありませんでした。CNNのレイヤー数や分類器のニューロン数を増やすと、せいぜい数パーセントの精度しか得られませんでした。

このことから、より高速なモデルを学習するために、軽量なネットワークアーキテクチャを試すことができるという考えに至りました。これは、モデルをモバイル機器で実行したい場合に特に重要です。

このモジュールでは、前のユニットでダウンロードした「Cats and Dogs」のデータセットを利用します。まず、データセットが利用可能かどうかを確認します。

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchinfo import summary
import os

from pytorchcv import train, display_dataset, train_long, load_cats_dogs_dataset, validate, common_transform

In [2]:
if not os.path.exists('data/kagglecatsanddogs_3367a.zip'):
    !wget -P data -q https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

dataset, train_loader, test_loader = load_cats_dogs_dataset()

/Users/shogo/miniforge3/envs/pytorch/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  warnings.warn(
/Users/shogo/miniforge3/envs/pytorch/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  warnings.warn(
/Users/shogo/miniforge3/envs/pytorch/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  warnings.warn(
/Users/shogo/miniforge3/envs/pytorch/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  warnings.warn(
/Users/shogo/miniforge3/envs/pytorch/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 

## MobileNet

前のユニットでは、画像分類のための**ResNet**アーキテクチャを見てきました。ResNetのより軽量なアナログは **MobileNet** で、これはいわゆる *Inverted Residual Blocks* を使用しています。事前に学習されたmobilenetをロードし、それがどのように動作するか見てみましょう。

In [3]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()
print(model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

Using cache found in /Users/shogo/.cache/torch/hub/pytorch_vision_v0.6.0


このモデルをデータセットに適用し、動作することを確認してみましょう。

In [4]:
sample_image = dataset[0][0].unsqueeze(0)
res = model(sample_image)
print(res[0].argmax())

tensor(281)


**演習：** MobileNetとフルスケールのResNetモデルのパラメータ数を比較する。

## MobileNetを使った転移学習

それでは、前のユニットと同じ伝達学習を、MobileNetを使って行ってみましょう。まず、モデルのすべてのパラメータを凍結します。

In [5]:
for x in model.parameters():
    x.requires_grad = False

そして、最終的な分類器を置き換えます。また、モデルをデフォルトのトレーニングデバイス（GPUまたはCPU）に転送します。

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.classifier = nn.Linear(1280, 2)
model = model.to(device)
summary(model, input_size=(1, 3, 244, 244))

Layer (type:depth-idx)                        Output Shape              Param #
MobileNetV2                                   --                        --
├─Sequential: 1-1                             [1, 1280, 8, 8]           --
│    └─ConvBNActivation: 2-1                  [1, 32, 122, 122]         --
│    │    └─Conv2d: 3-1                       [1, 32, 122, 122]         (864)
│    │    └─BatchNorm2d: 3-2                  [1, 32, 122, 122]         (64)
│    │    └─ReLU6: 3-3                        [1, 32, 122, 122]         --
│    └─InvertedResidual: 2-2                  [1, 16, 122, 122]         --
│    │    └─Sequential: 3-4                   [1, 16, 122, 122]         (896)
│    └─InvertedResidual: 2-3                  [1, 24, 61, 61]           --
│    │    └─Sequential: 3-5                   [1, 24, 61, 61]           (5,136)
│    └─InvertedResidual: 2-4                  [1, 24, 61, 61]           --
│    │    └─Sequential: 3-6                   [1, 24, 61, 61]           (8,832)
│ 

では、実際にトレーニングをしてみましょう。

In [7]:
train_long(model, train_loader, test_loader, loss_fn=torch.nn.CrossEntropyLoss(), epochs=1, print_freq=90)

Epoch 0, minibatch 0: train acc = 0.40625, train loss = 0.02217177301645279


## 覚えておくべき重要点

MobileNetの精度は、VGG-16とほぼ同じで、本格的なResNetよりもわずかに低いことがわかります。

MobileNetやResNet-18のような小型モデルの最大の利点は、モバイル機器で利用できることです。[こちら](https://pytorch.org/mobile/android/)はAndroid端末でResNet-18を使用した公式の例であり、[こちら](https://heartbeat.fritz.ai/pytorch-mobile-image-classification-on-android-5c0cfb774c5b)はMobileNetを使用した同様の例です。